In [1]:
import pandas as pd
from datetime import datetime
#!pip install quandl
import quandl
import warnings
warnings.filterwarnings("ignore")

In [2]:
file = ['dow', 'euro_stoxx50', 'nasdaq', 'nikkei', 'oro', 'petroleo', 'shangai_stock_exchange', 'sp_500']
data = []
for i in file:
    data.append(pd.read_csv('./data/' + i + '.csv'))
file.insert(0, 'date')

In [3]:
for i in data:
    print(i.info(), '\n \n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 7 columns):
Fecha       1125 non-null object
Cierre      1125 non-null float64
Apertura    1125 non-null float64
Máximo      1125 non-null float64
Mínimo      1125 non-null float64
Vol.        1125 non-null object
% var.      1125 non-null object
dtypes: float64(4), object(3)
memory usage: 61.6+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1539 entries, 0 to 1538
Data columns (total 7 columns):
Fecha       1539 non-null object
Cierre      1539 non-null object
Apertura    1539 non-null object
Máximo      1539 non-null object
Mínimo      1539 non-null object
Vol.        1539 non-null object
% var.      1539 non-null object
dtypes: object(7)
memory usage: 84.2+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 7 columns):
Fecha       1512 non-null object
Cierre      1512 non-null float64
Apertura    1512 non-null float64
Máx

In [4]:
def price(data):
    try:
        amount = [float(i.replace(',','')) for i in data.Cierre]
        df = pd.DataFrame({'date': pd.to_datetime(data.Fecha),
                      'amount': amount})
    except:
        df = pd.DataFrame({'date':pd.to_datetime(data.Fecha),
                      'amount': data.Cierre.astype('float')})
    return df

def variance(data):
    var = [float(i.replace('%','')) for i in data.iloc[:,6]]
    df = pd.DataFrame({'date': pd.to_datetime(data.Fecha),
                      'var': var})
    return df

def generate_df(l, names, ind, rename = True):
    inner = pd.merge(l[0], l[1], how ='inner', on = 'date', left_on = None, right_on = None,
         left_index = True, right_index = True, sort = True)
    for i in range(2, len(l)):
        inner = pd.merge(inner, l[i], how ='inner', on = 'date', left_on = None, right_on = None,
                 left_index = True, right_index = True, sort = True)
    inner = pd.DataFrame(inner)
    if rename:
        inner.columns = names
        for i in range(1, len(names)):
            inner.rename(columns = {names[i]: names[i] + ind}, inplace = True)
    return inner.sort_values('date')

def get_info_bit(start, end, key):
    inf = quandl.get(key, start_date = start,end_date = end)
    return inf

In [5]:
amount = []
var = []
for i in data:
    amount.append(price(i))
    var.append(variance(i))

In [6]:
amount[0].head(5)

,date,amount
0,2020-12-31,55.50
1,2020-12-30,55.47
2,2020-12-29,54.33
3,2020-12-28,54.29
4,2020-12-24,54.79


In [7]:
var[0].head(5)

,date,var
0,2020-12-31,0.05
1,2020-12-30,2.10
2,2020-12-29,0.07
3,2020-12-28,-0.91
4,2020-12-24,0.07


In [8]:
for i in amount:
    print('max: ', i.date.max(), '\t min: ', i.date.min())

max:  2020-12-31 00:00:00 	 min:  2015-01-04 00:00:00
max:  2020-12-31 00:00:00 	 min:  2015-01-04 00:00:00
max:  2020-12-31 00:00:00 	 min:  2015-01-04 00:00:00
max:  2020-12-30 00:00:00 	 min:  2015-01-04 00:00:00
max:  2021-01-01 00:00:00 	 min:  2015-01-01 00:00:00
max:  2020-12-31 00:00:00 	 min:  2015-01-01 00:00:00
max:  2020-12-31 00:00:00 	 min:  2015-01-04 00:00:00
max:  2020-12-31 00:00:00 	 min:  2015-01-04 00:00:00


In [9]:
amount = generate_df(amount, file, '(USD)')
amount.head()

,date,dow(USD),euro_stoxx50(USD),nasdaq(USD),nikkei(USD),oro(USD),petroleo(USD),shangai_stock_exchange(USD),sp_500(USD)
1063,2015-01-04,47.90,3009.28,65.94,16819.24,1134.55,52.23,2.95,2136.73
1042,2015-01-05,51.69,3100.76,66.93,17177.21,1298.90,52.48,2.86,2163.26
1022,2015-01-06,52.26,3317.52,67.77,18360.54,1237.30,53.20,2.96,2246.19
1000,2015-01-07,51.39,3273.11,67.72,19364.67,1329.00,48.86,3.08,2275.32
957,2015-01-09,41.66,3472.58,70.82,19633.75,1329.90,48.66,3.03,2385.26


In [10]:
var = generate_df(var, file, '(%)')
var.head()

,date,dow(%),euro_stoxx50(%),nasdaq(%),nikkei(%),oro(%),petroleo(%),shangai_stock_exchange(%),sp_500(%)
1063,2015-01-04,-0.17,1.85,-0.72,0.50,-0.04,0.21,-0.24,-1.24
1042,2015-01-05,1.35,1.57,2.37,1.61,0.75,-0.08,0.00,1.11
1022,2015-01-06,0.36,0.08,0.24,0.47,-6.22,0.51,0.00,0.22
1000,2015-01-07,0.43,0.60,0.77,0.33,0.90,2.39,0.49,0.28
957,2015-01-09,-4.80,-0.26,0.16,0.15,0.05,-0.39,0.20,0.84


In [11]:
quandl.ApiConfig.api_key = "yRoEr9d428JVxLEqdo77"
start="2015-01-01"
end="2021-01-01"

In [12]:
bit = {'BCHAIN-TOTBC':'Total de bitcoins disponibles',
      'BCHAIN-MKPRU':'Precio del bitcoin en dolares',
      'BCHAIN-AVBLS':'Tamaño promedio del bloque',
      'BCHAIN-MIREV':'Recompensas por bloques de monedas y tarifas de transacción pagadas a los mineros',
      'BCHAIN-HRATE':'Número estimado de Tera de hashes por segundo que realizan todos los mineros',
      'BCHAIN-ATRCT':'Tiempo medio de confirmación de transacción',
      'BCHAIN-NTRBL':'Número de transacciones de Bitcoin por bloque',
      'BCHAIN-NADDU':'Número total de un Bitcoin único'}
bitcoin_var = bit.keys()
bitcoin_var

dict_keys(['BCHAIN-TOTBC', 'BCHAIN-MKPRU', 'BCHAIN-AVBLS', 'BCHAIN-MIREV', 'BCHAIN-HRATE', 'BCHAIN-ATRCT', 'BCHAIN-NTRBL', 'BCHAIN-NADDU'])

In [13]:
bitcoin_data = []
for i in bitcoin_var:
    bitcoin_data.append(pd.read_csv('./data/' + i + '.csv'))
for i in range(0, len(bitcoin_data)):
    bitcoin_data[i].rename(columns = {'Date':'date'}, inplace = True)

In [14]:
bitcoin_var = list(bitcoin_var)
bitcoin_var.insert(0, 'date')

In [15]:
bitcoin_data[0].head(5)

,date,Value
0,2021-01-09,18594850.00
1,2021-01-08,18593975.00
2,2021-01-07,18592875.00
3,2021-01-06,18591781.25
4,2021-01-05,18590687.50


In [16]:
for i in bitcoin_data:
    print(i.info(), '\n \n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4389 entries, 0 to 4388
Data columns (total 2 columns):
date     4389 non-null object
Value    4389 non-null float64
dtypes: float64(1), object(1)
memory usage: 68.7+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4391 entries, 0 to 4390
Data columns (total 2 columns):
date     4391 non-null object
Value    4391 non-null float64
dtypes: float64(1), object(1)
memory usage: 68.7+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4390 entries, 0 to 4389
Data columns (total 2 columns):
date     4390 non-null object
Value    4390 non-null float64
dtypes: float64(1), object(1)
memory usage: 68.7+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4390 entries, 0 to 4389
Data columns (total 2 columns):
date     4390 non-null object
Value    4390 non-null float64
dtypes: float64(1), object(1)
memory usage: 68.7+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4389 entries, 0 to 4388
Data colum

In [17]:
bitcoin_data = generate_df(bitcoin_data, bitcoin_var, '')
bitcoin_data.rename(columns = {'BCHAIN-MKPRU':'BCHAIN-MKPRU (USD)', 'BCHAIN-MIREV':'BCHAIN-MIREV (USD)',
                   'BCHAIN-ATRCT':'BCHAIN-ATRCT (SEG)'}, inplace = True)
bitcoin_data['date'] = pd.to_datetime(bitcoin_data.date)
bitcoin_data.head(5)

,date,BCHAIN-TOTBC,BCHAIN-MKPRU (USD),BCHAIN-AVBLS,BCHAIN-MIREV (USD),BCHAIN-HRATE,BCHAIN-ATRCT (SEG),BCHAIN-NTRBL,BCHAIN-NADDU
3503,2011-06-07,6462050.0,31.50,0.026287,293220.0711,5.160439,0.0,49.0,15812.0
3502,2011-06-08,6471200.0,35.00,0.025533,335412.0000,5.978214,0.0,56.0,20730.0
3501,2011-06-09,6481800.0,30.00,0.019570,396795.0000,6.373001,0.0,50.0,22272.0
3500,2011-06-10,6493100.0,24.99,0.018799,349080.0000,6.542196,0.0,58.0,21061.0
3499,2011-06-11,6504700.0,24.50,0.019836,298405.5900,6.711391,0.0,64.0,20896.0


In [18]:
#file = ['eur_usd','cny_usd','jpy_usd','gbp_usd']
file = ['eur_usd','cny_usd','gbp_usd']
data = []
for i in file:
    data.append(pd.read_csv('./data/' + i + '.csv'))
file.insert(0, 'date')

In [19]:
for i in data:
    print(i.info(), '\n \n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 6 columns):
Fecha       1567 non-null object
Cierre      1567 non-null float64
Apertura    1567 non-null float64
Máximo      1567 non-null float64
Mínimo      1567 non-null float64
% var.      1567 non-null object
dtypes: float64(4), object(2)
memory usage: 73.5+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 6 columns):
Fecha       1567 non-null object
Cierre      1567 non-null float64
Apertura    1567 non-null float64
Máximo      1567 non-null float64
Mínimo      1567 non-null float64
% var.      1567 non-null object
dtypes: float64(4), object(2)
memory usage: 73.5+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 6 columns):
Fecha       1567 non-null object
Cierre      1567 non-null float64
Apertura    1567 non-null float64
Máximo      1567 non-null float64
Mínimo      1567 no

In [20]:
general = []
for i in data:
    general.append(price(i))
general = generate_df(general, file, '')
general.head()

,date,eur_usd,cny_usd,gbp_usd
1566,2015-01-01,1.2104,0.1611,1.5545
1502,2015-01-04,1.0764,0.1613,1.4823
1480,2015-01-05,1.1202,0.1612,1.5141
1459,2015-01-06,1.0928,0.1613,1.5201
1437,2015-01-07,1.1054,0.1613,1.5618


In [21]:
complete = [amount, bitcoin_data, general]
complete = generate_df(complete, file, '', False)
complete.head()

,date,dow(USD),euro_stoxx50(USD),nasdaq(USD),nikkei(USD),oro(USD),petroleo(USD),shangai_stock_exchange(USD),sp_500(USD),BCHAIN-TOTBC,BCHAIN-MKPRU (USD),BCHAIN-AVBLS,BCHAIN-MIREV (USD),BCHAIN-HRATE,BCHAIN-ATRCT (SEG),BCHAIN-NTRBL,BCHAIN-NADDU,eur_usd,cny_usd,gbp_usd
1063,2015-01-04,47.90,3009.28,65.94,16819.24,1134.55,52.23,2.95,2136.73,16857300.0,8343.455000,1.034792,1.515466e+07,2.043482e+07,8.850000,1271.209790,462086.0,1.0719,0.1453,1.2677
1042,2015-01-05,51.69,3100.76,66.93,17177.21,1298.90,52.48,2.86,2163.26,16898562.5,11430.181667,0.976834,2.548563e+07,2.690962e+07,8.033333,983.466667,524246.0,1.0488,0.1442,1.2322
1022,2015-01-06,52.26,3317.52,67.77,18360.54,1237.30,53.20,2.96,2246.19,16935862.5,8662.378333,0.830548,1.445054e+07,2.289248e+07,7.700000,1352.015038,418187.0,1.0769,0.1453,1.2658
1000,2015-01-07,51.39,3273.11,67.72,19364.67,1329.00,48.86,3.08,2275.32,16977875.0,8340.748333,0.798910,1.376253e+07,2.614693e+07,9.166667,1181.335766,503405.0,1.0623,0.1449,1.2295
957,2015-01-09,41.66,3472.58,70.82,19633.75,1329.90,48.66,3.03,2385.26,17059087.5,7130.541667,0.448727,1.509175e+07,3.511234e+07,7.866667,879.902439,356715.0,1.0885,0.1449,1.2867


In [22]:
complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1125 entries, 1063 to 0
Data columns (total 20 columns):
date                           1125 non-null datetime64[ns]
dow(USD)                       1125 non-null float64
euro_stoxx50(USD)              1125 non-null float64
nasdaq(USD)                    1125 non-null float64
nikkei(USD)                    1125 non-null float64
oro(USD)                       1125 non-null float64
petroleo(USD)                  1125 non-null float64
shangai_stock_exchange(USD)    1125 non-null float64
sp_500(USD)                    1125 non-null float64
BCHAIN-TOTBC                   1125 non-null float64
BCHAIN-MKPRU (USD)             1125 non-null float64
BCHAIN-AVBLS                   1125 non-null float64
BCHAIN-MIREV (USD)             1125 non-null float64
BCHAIN-HRATE                   1125 non-null float64
BCHAIN-ATRCT (SEG)             1125 non-null float64
BCHAIN-NTRBL                   1125 non-null float64
BCHAIN-NADDU                   1125 no

In [23]:
print('Max date: ', complete.date.max(),'\t Min date:', complete.date.min())

Max date:  2020-12-31 00:00:00 	 Min date: 2015-01-04 00:00:00


In [24]:
def completa(df, start, end):
    names = list(df.columns)
    date = pd.date_range(start = start, end = end).difference(df.iloc[:,0])
    new = pd.DataFrame(columns = names)
    new['date'] = date
    new = pd.concat([df, new]).sort_values('date')
    return new

import math
def media(l):
    if not math.isnan(l[0]):
        if not math.isnan(l[2]):
            return (l[0] + l[2])/2
        else:
            return l[0]
    elif not math.isnan(l[2]):
        return l[2]
        
def rellena(df):
    nan = list(df.iloc[:,1].isnull())
    idx = []
    for i, j in enumerate(nan):
        if j:
            idx.append(i)
    for col in range(1, len(df.columns)):
        for i in idx:
            df.iloc[i,col] = media(list(df.iloc[[i - 1, i, i + 1],col]))
    return df

In [25]:
df_aux = completa(complete, '2015-01-04', '2020-12-31')
final = pd.concat([complete, df_aux]).sort_values('date').drop_duplicates().reset_index()
final.drop(['index'], axis=1, inplace = True)
final

,date,dow(USD),euro_stoxx50(USD),nasdaq(USD),nikkei(USD),oro(USD),petroleo(USD),shangai_stock_exchange(USD),sp_500(USD),BCHAIN-TOTBC,BCHAIN-MKPRU (USD),BCHAIN-AVBLS,BCHAIN-MIREV (USD),BCHAIN-HRATE,BCHAIN-ATRCT (SEG),BCHAIN-NTRBL,BCHAIN-NADDU,eur_usd,cny_usd,gbp_usd
0,2015-01-04,47.90,3009.28,65.94,16819.24,1134.55,52.23,2.95,2136.73,16857300.00,8343.455000,1.034792,1.515466e+07,2.043482e+07,8.850000,1271.209790,462086.0,1.0719,0.1453,1.2677
1,2015-01-05,51.69,3100.76,66.93,17177.21,1298.90,52.48,2.86,2163.26,16898562.50,11430.181667,0.976834,2.548563e+07,2.690962e+07,8.033333,983.466667,524246.0,1.0488,0.1442,1.2322
2,2015-01-06,52.26,3317.52,67.77,18360.54,1237.30,53.20,2.96,2246.19,16935862.50,8662.378333,0.830548,1.445054e+07,2.289248e+07,7.700000,1352.015038,418187.0,1.0769,0.1453,1.2658
3,2015-01-07,51.39,3273.11,67.72,19364.67,1329.00,48.86,3.08,2275.32,16977875.00,8340.748333,0.798910,1.376253e+07,2.614693e+07,9.166667,1181.335766,503405.0,1.0623,0.1449,1.2295
4,2015-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-01-09,41.66,3472.58,70.82,19633.75,1329.90,48.66,3.03,2385.26,17059087.50,7130.541667,0.448727,1.509175e+07,3.511234e+07,7.866667,879.902439,356715.0,1.0885,0.1449,1.2867
6,2015-01-10,42.98,3642.11,68.11,18800.50,1328.50,46.46,3.06,2328.95,17099587.50,6713.488000,0.490980,1.268641e+07,3.855979e+07,7.450000,922.980892,343364.0,1.1213,0.1469,1.2881
7,2015-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-01-12,53.42,3478.41,71.05,19860.03,1347.50,48.59,3.00,2437.92,17179137.50,8143.148333,0.539321,1.743165e+07,5.027718e+07,9.833333,990.629412,395733.0,1.1842,0.1487,1.3214
9,2015-01-13,43.61,2993.73,68.50,16497.85,1307.70,49.83,2.72,2169.18,16750612.50,17737.111667,1.042035,5.260019e+07,1.694649e+07,8.433333,2166.148352,877464.0,1.1189,0.1499,1.3027


In [26]:
final2 = rellena(final)
final2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2189 entries, 0 to 2188
Data columns (total 20 columns):
date                           2189 non-null datetime64[ns]
dow(USD)                       2189 non-null float64
euro_stoxx50(USD)              2189 non-null float64
nasdaq(USD)                    2189 non-null float64
nikkei(USD)                    2189 non-null float64
oro(USD)                       2189 non-null float64
petroleo(USD)                  2189 non-null float64
shangai_stock_exchange(USD)    2189 non-null float64
sp_500(USD)                    2189 non-null float64
BCHAIN-TOTBC                   2189 non-null float64
BCHAIN-MKPRU (USD)             2189 non-null float64
BCHAIN-AVBLS                   2189 non-null float64
BCHAIN-MIREV (USD)             2189 non-null float64
BCHAIN-HRATE                   2189 non-null float64
BCHAIN-ATRCT (SEG)             2189 non-null float64
BCHAIN-NTRBL                   2189 non-null float64
BCHAIN-NADDU                   2189 no

In [27]:
cols = list(final2.columns)
cols.remove('BCHAIN-MKPRU (USD)')
cols.append('BCHAIN-MKPRU (USD)')
final2 = final2[cols] 
final2.head()

,date,dow(USD),euro_stoxx50(USD),nasdaq(USD),nikkei(USD),oro(USD),petroleo(USD),shangai_stock_exchange(USD),sp_500(USD),BCHAIN-TOTBC,BCHAIN-AVBLS,BCHAIN-MIREV (USD),BCHAIN-HRATE,BCHAIN-ATRCT (SEG),BCHAIN-NTRBL,BCHAIN-NADDU,eur_usd,cny_usd,gbp_usd,BCHAIN-MKPRU (USD)
0,2015-01-04,47.900,3009.280,65.94,16819.24,1134.55,52.23,2.950,2136.73,16857300.00,1.034792,1.515466e+07,2.043482e+07,8.850000,1271.209790,462086.0,1.0719,0.1453,1.2677,8343.455000
1,2015-01-05,51.690,3100.760,66.93,17177.21,1298.90,52.48,2.860,2163.26,16898562.50,0.976834,2.548563e+07,2.690962e+07,8.033333,983.466667,524246.0,1.0488,0.1442,1.2322,11430.181667
2,2015-01-06,52.260,3317.520,67.77,18360.54,1237.30,53.20,2.960,2246.19,16935862.50,0.830548,1.445054e+07,2.289248e+07,7.700000,1352.015038,418187.0,1.0769,0.1453,1.2658,8662.378333
3,2015-01-07,51.390,3273.110,67.72,19364.67,1329.00,48.86,3.080,2275.32,16977875.00,0.798910,1.376253e+07,2.614693e+07,9.166667,1181.335766,503405.0,1.0623,0.1449,1.2295,8340.748333
4,2015-01-08,46.525,3372.845,69.27,19499.21,1329.45,48.76,3.055,2330.29,17018481.25,0.623819,1.442714e+07,3.062963e+07,8.516667,1030.619103,430060.0,1.0754,0.1449,1.2581,7735.645000


In [30]:
final2.to_csv("data.csv", index = False)

In [29]:
file = ['dow', 'euro_stoxx50', 'nasdaq', 'nikkei', 'oro', 'petroleo', 'shangai_stock_exchange', 'sp_500',
       'eur_usd','cny_usd','gbp_usd']
bitcoin_var = list(bit.keys())
for i in file:
    print(i + ':\t',pd.read_csv('./data/' + i + '.csv').shape[0])
for i in bitcoin_var:
    print(i + ':\t',pd.read_csv('./data/' + i + '.csv').shape[0])
    #print(i + ':\t',get_info_bit(start, end, i).reset_index().shape[0])

dow:	 1125
euro_stoxx50:	 1539
nasdaq:	 1512
nikkei:	 1494
oro:	 1588
petroleo:	 1583
shangai_stock_exchange:	 1434
sp_500:	 1511
eur_usd:	 1567
cny_usd:	 1567
gbp_usd:	 1567
BCHAIN-TOTBC:	 4389
BCHAIN-MKPRU:	 4391
BCHAIN-AVBLS:	 4390
BCHAIN-MIREV:	 4390
BCHAIN-HRATE:	 4389
BCHAIN-ATRCT:	 3504
BCHAIN-NTRBL:	 4390
BCHAIN-NADDU:	 4389
